In [10]:
import pandas as pd
import numpy as np 

Task 1


In [11]:
def read_from_file(dataFile, N):
    df = pd.read_csv(dataFile, sep = ',', nrows=N)
    return df

number_of_rows = 20000
data = read_from_file('data_000637.txt', number_of_rows)

print(data)

       HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0         1     0          123  3869200167        2374        26
1         1     0          124  3869200167        2374        27
2         1     0           63  3869200167        2553        28
3         1     0           64  3869200167        2558        19
4         1     0           64  3869200167        2760        25
...     ...   ...          ...         ...         ...       ...
19995     1     0           27  3869200366        2513        29
19996     1     0           63  3869200366        2517         6
19997     1     0           32  3869200366        2519         5
19998     1     0           17  3869200366        2522        21
19999     1     0           64  3869200366        2522         0

[20000 rows x 6 columns]


Task 2


In [12]:
#These functions do the same, but the first function was done according to the hints
'''
def estimate_BX_in_orbit(dataFrame, orbit_col, counter_col):
    max_BX_values = np.array([])   
    prev_orbit, prev_counter = None, None
    
    for index, row in dataFrame.iterrows():
        orbit, counter = row[orbit_col], row[counter_col]
        
        if ((orbit != prev_orbit) and (prev_counter is not None)):
    
            max_BX_values = np.append(max_BX_values, prev_counter)
        
        prev_orbit, prev_counter = orbit, counter
    
    estimate_BX = np.max(max_BX_values)
    return estimate_BX


est_BX_in_orbit = estimate_BX_in_orbit(data, 'ORBIT_CNT', 'BX_COUNTER')

print(f'The estimated number of BX in an ORBIT: {est_BX_in_orbit} \n')
'''

def estimate_BX_in_orbit(dataFrame, counter_col):
    max_BX = np.max(dataFrame[counter_col])
    return max_BX


est_BX_in_orbit = estimate_BX_in_orbit(data, 'BX_COUNTER')

print(f'The estimated number of BX in an ORBIT: {est_BX_in_orbit} \n')


The estimated number of BX in an ORBIT: 3563 



Task 3


In [13]:
BX_UNIT_NS = 25  # ns
TDC_MEAS_UNIT_NS = 25 / 30  # ns
BX_COUNTER_MAX = est_BX_in_orbit  # Max value of BX_COUNTER before resetting to 0

data['absolute_time_ns'] = (
    data['ORBIT_CNT'] * (BX_COUNTER_MAX + 1) * BX_UNIT_NS +  # Absolute orbit time
    data['BX_COUNTER'] * BX_UNIT_NS +  # Absolute BX_COUNTER time
    data['TDC_MEAS'] * TDC_MEAS_UNIT_NS  # Absolute TDC_MEAS time
)

# Starting time and offset all times by it
starting_time = data.loc[0, 'absolute_time_ns'] #0 is the row, 'absolute_time_ns' is the column just made
data['absolute_time_ns'] -= starting_time

#Time series
data['absolute_time_series'] = pd.to_datetime(data['absolute_time_ns'], unit='ns') #1970-01-01 is the Unix epoch reference

print(data[['ORBIT_CNT', 'BX_COUNTER', 'TDC_MEAS', 'absolute_time_ns', 'absolute_time_series']])


        ORBIT_CNT  BX_COUNTER  TDC_MEAS  absolute_time_ns  \
0      3869200167        2374        26      0.000000e+00   
1      3869200167        2374        27      8.125000e-01   
2      3869200167        2553        28      4.476625e+03   
3      3869200167        2558        19      4.594125e+03   
4      3869200167        2760        25      9.649125e+03   
...           ...         ...       ...               ...   
19995  3869200366        2513        29      1.773438e+07   
19996  3869200366        2517         6      1.773446e+07   
19997  3869200366        2519         5      1.773451e+07   
19998  3869200366        2522        21      1.773460e+07   
19999  3869200366        2522         0      1.773458e+07   

               absolute_time_series  
0     1970-01-01 00:00:00.000000000  
1     1970-01-01 00:00:00.000000000  
2     1970-01-01 00:00:00.000004476  
3     1970-01-01 00:00:00.000004594  
4     1970-01-01 00:00:00.000009649  
...                             ...  
1

Task 4


In [14]:
# Find the duration of the data taking
duration = data['absolute_time_series'].max() - data['absolute_time_series'].min()
print(f'The duration of the data taking is: {duration}')
# Convert the duration to hours, minutes, and seconds
hours = duration.components.hours
minutes = duration.components.minutes
seconds = duration.components.seconds

# Display the duration in hours, minutes, and seconds
print(f"And represented in H,M and S: \n Duration: {hours} hours, {minutes} minutes, {seconds} seconds")

The duration of the data taking is: 0 days 00:00:00.017734595
And represented in H,M and S: 
 Duration: 0 hours, 0 minutes, 0 seconds


Task 5

In [15]:
top_3_channels = data.groupby('TDC_CHANNEL')['BX_COUNTER'].sum().nlargest(3)

print(f'The TDC_CHANNELS with most noise is: \n {top_3_channels} \n')


The TDC_CHANNELS with most noise is: 
 TDC_CHANNEL
139    2991203
64     1834885
63     1753201
Name: BX_COUNTER, dtype: int64 



Task 6


In [16]:
non_empty_orbits = data['ORBIT_CNT'].count()

print(f'The number of non-empty orbits is: {non_empty_orbits}')

The number of non-empty orbits is: 20000


Task 7


In [17]:
unique_orbits_with_TDC139 = data[data['TDC_CHANNEL'] == 139]['ORBIT_CNT'].nunique()
print(f'The number of unique orbits that has a measurement from the TDC_CHANNEL = 139 is: {unique_orbits_with_TDC139}')


The number of unique orbits that has a measurement from the TDC_CHANNEL = 139 is: 200


Task 8

In [18]:
FPGA0_counts = data[data['FPGA'] == 0]['TDC_CHANNEL'].value_counts()
FPGA1_counts = data[data['FPGA'] == 1]['TDC_CHANNEL'].value_counts()

print(f'The series for the FPGA0: \n {FPGA0_counts} \n')
print(f'The series for the FPGA1: \n {FPGA1_counts} \n')

The series for the FPGA0: 
 139    1171
64      975
63      961
61      726
62      711
       ... 
106       6
107       6
98        5
101       3
67        2
Name: TDC_CHANNEL, Length: 118, dtype: int64 

The series for the FPGA1: 
 139    522
2      479
1      435
4      393
3      362
      ... 
78       1
93       1
86       1
76       1
91       1
Name: TDC_CHANNEL, Length: 125, dtype: int64 

